In [3]:
# importing the libraries
import numpy as np
import pandas as pd 
import pandas as pd

In [21]:
# definging layer class
class Layer:
    def __init__(self, row, col):
        self.input = None
        self.output = None
        self.init(row, col)
    
    def init(self, row, col):
        self.weights = np.random.randn(row, col)
        self.biases = np.zeros(row)
    
    def activation(self, outputs):
        outputs = np.clip(outputs, -500, 500)
        return 1/(1 + np.exp(-outputs))
    
    def feedForward(self, input):
        # storing the inputs as outputs for using in backpropagations
        self.input = input
        output = np.dot(input, self.weights.T) + self.biases
        self.output = self.activation(output)
        return self.output
    
    def backPropagation(self, errorWRTout, learningR):
        # Backpropagation for the output layer
        errorWRTnet = self.output * (1 - self.output) * errorWRTout
        errorWRTweight = np.transpose([errorWRTnet]) * self.input
        errorWRTinput = np.dot(errorWRTnet, self.weights)
        self.weights -= learningR * errorWRTweight
        self.biases -= learningR * errorWRTnet
        return errorWRTinput

In [22]:
class NeuralNetwork:
    def __init__(self, sizes):
        self.sizes = sizes
        self.layers = []
        self.epochs = 1
        self.learningR = .001
    
    def initLayers(self):
        # Initializing all layers
        self.layers.append(Layer(self.sizes[0], 1))
        for i in range(1, len(self.sizes)):
            layer = Layer(self.sizes[i], self.sizes[i-1])
            self.layers.append(layer)

    def setLearningRate(self, learningR):
        self.learningR = learningR

    def setEpoch(self, epochs):
        self.epochs = epochs
    
    # function to calculate mean squared error
    def claculateError(self, output, target):
        error = np.square(np.subtract(target, output)).mean()
        return error
    
    def predict(self, input):
        output = input
        for layer in self.layers[1:]:
            output = layer.feedForward(output)
        return output
    
    def updateWeight(self, errorWRTout):
        for layer in reversed(self.layers[1:]):
            errorWRTout = layer.backPropagation(errorWRTout, self.learningR)

    def trainOneEpoch(self, inputs, targets):
        for input, target in zip(inputs, targets):
            output = self.predict(input)
            errorWRTout = output - target
            self.updateWeight(errorWRTout)

            
    def trainModel(self, inputs, targets):
        for i in range(self.epochs):
            self.trainOneEpoch(inputs, targets)

In [16]:
# loading the training dataset
df = pd.read_csv('./dataset/mnist_train.csv')

In [23]:
# seperating label and features data
labels = df.iloc[:, 0].values
inputs = df.iloc[:, 1:].values

# converting labels to one hot
oneHots = np.zeros((labels.size, 10))
rows = np.arange(labels.size)
oneHots[rows, labels] = 1

In [26]:
# main part of building the model
network = NeuralNetwork([784, 50, 50, 50, 50, 10])
network.initLayers()
network.setEpoch(20)
network.setLearningRate(.01)
network.trainModel(inputs, oneHots)

In [32]:
# running the training again manually
network.setLearningRate(10)
network.trainModel(inputs, oneHots)

In [ ]:
# printing error of the model
for input, label in zip(inputs, oneHots):
    output = network.predict(input)
    print(network.claculateError(output, label))

In [28]:
# import dataset for testing
df2 = pd.read_csv('./dataset/mnist_test.csv')

In [29]:
# seperating the features and labels
labels2 = df2.iloc[:100, 0].values
inputs2 = df2.iloc[:100, 1:].values

In [ ]:
# testing the model
for input, label in zip(inputs2, labels2):
    output = network.predict(input)
    print('label:', label, 'result:', np.argmax(output))
